In [6]:
import torch
import matplotlib.pyplot as plt
import wandb
#import torch.utils.tensorboard as tutb
import os

def plot_model_weights_histogram(model_path):
    # Load the model's state dictionary
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))

    # Extract weights from each layer and combine them into a single list
    weights = []
    for param_tensor in state_dict:
        if "weight" in param_tensor:  # Look only for weight tensors
            weights.extend(state_dict[param_tensor].cpu().numpy().flatten())

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(weights, bins=50, alpha=0.75)
    plt.xlabel('Weight values')
    plt.ylabel('Frequency')
    plt.title('Histogram of Model Weights')
    plt.grid(True)
    plt.show()


In [ ]:
# Example usage
plot_model_weights_histogram('model_weights/RAdam_20241113_004451/RAdam_epoch_3_lr=0.01_weight_decay=0.pt')

C:\Users\hz283\AppData\Local\Temp\ipykernel_60800\2814157227.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.devic